# Analyse de proximité entre participants (cours + discussions)

## Connexion à MongoDB

In [2]:
# Connexion à la base MongoDB locale
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['MOOC']
collection = db['forum']

print("Connexion à MongoDB réussie ✅")

Connexion à MongoDB réussie ✅


## Sélection d’un participant de référence

In [3]:
# Participant choisi (à personnaliser)
participant = "SC91"

print(f"Participant choisi : {participant}")

Participant choisi : SC91


## Extraction de ses interactions (cours & threads)

In [4]:
# Récupérer les threads créés et commentés par le participant
threads = collection.find({ "content.username": participant })
comments = collection.find({ "content.children.username": participant })

# Construire les ensembles de cours et threads
cours_participant = set()
threads_participant = set()

for doc in threads:
    cours_participant.add(doc['content']['course_id'])
    threads_participant.add(doc['content']['id'])

for doc in comments:
    cours_participant.add(doc['content']['course_id'])
    threads_participant.add(doc['content']['id'])

# Vérification
print(f"Cours suivis par {participant} : {cours_participant}")
print(f"Threads touchés par {participant} : {threads_participant}")

Cours suivis par SC91 : {'CNAM/01002/Trimestre_1_2014'}
Threads touchés par SC91 : {'52ef6120122bb64c4f000924', '52fcaedb4b4451f4ad000065', '52ef611a38bffc84380008e9'}


## Récupération de tous les autres participants

In [5]:
# Liste de tous les participants uniques
usernames = collection.distinct("content.username")

# Exclure le participant de référence
usernames = [u for u in usernames if u != participant]

print(f"{len(usernames)} autres participants à comparer")

36241 autres participants à comparer


## Fonction de similarité (Jaccard)

In [6]:
# Calcul de la similarité Jaccard entre 2 ensembles
def jaccard(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

## Calcul des similarités pour chaque participant

In [7]:
from collections import defaultdict

# Dictionnaires pour stocker cours et threads par user
user_cours = defaultdict(set)
user_threads = defaultdict(set)

# On récupère TOUS les threads et commentaires d’un coup
docs = collection.find({
    "content.username": {"$in": usernames}
})

for doc in docs:
    user = doc['content']['username']
    user_cours[user].add(doc['content']['course_id'])
    user_threads[user].add(doc['content']['id'])

# Pareil pour les commentaires dans children (si applicable)
docs_comments = collection.find({
    "content.children.username": {"$in": usernames}
})

for doc in docs_comments:
    children = doc['content'].get('children', [])
    for child in children:
        user = child['username']
        user_cours[user].add(doc['content']['course_id'])
        user_threads[user].add(doc['content']['id'])

# Calcul des similarités (en mémoire = rapide)
resultats = []

for other in usernames:
    cours_other = user_cours[other]
    threads_other = user_threads[other]

    score_cours = jaccard(cours_participant, cours_other)
    score_threads = jaccard(threads_participant, threads_other)
    score_moyen = (score_cours + score_threads) / 2

    resultats.append((other, score_cours, score_threads, score_moyen))

## Tri et affichage des résultats

In [8]:
# Trier les participants par similarité décroissante
resultats = sorted(resultats, key=lambda x: x[3], reverse=True)

# Afficher les 10 plus proches
print(f"Top 10 des participants les plus proches de {participant} :\n")
for r in resultats[:10]:
    print(f"{r[0]} | Similarité cours={r[1]:.2f} | threads={r[2]:.2f} | moyenne={r[3]:.2f}")

Top 10 des participants les plus proches de SC91 :

angel13119 | Similarité cours=1.00 | threads=0.33 | moyenne=0.67
CELINE95 | Similarité cours=1.00 | threads=0.25 | moyenne=0.62
Kduce | Similarité cours=1.00 | threads=0.20 | moyenne=0.60
Robert-78 | Similarité cours=1.00 | threads=0.20 | moyenne=0.60
sandiemoreau | Similarité cours=1.00 | threads=0.20 | moyenne=0.60
Mouss | Similarité cours=1.00 | threads=0.17 | moyenne=0.58
AlKines | Similarité cours=1.00 | threads=0.14 | moyenne=0.57
FabienneC | Similarité cours=1.00 | threads=0.09 | moyenne=0.55
0Livi3r | Similarité cours=1.00 | threads=0.00 | moyenne=0.50
3pommes12 | Similarité cours=1.00 | threads=0.00 | moyenne=0.50
